# Guided Project: Answering Business Questions using SQL

* **About The Database**
The database represents a digital media store,iTunes Library, including tables for artists, albums, media tracks, invoices and customers. For details about the database [Chinook database](https://github.com/lerocha/chinook-database)

##### Connecting the Jupyter notebook to the chinook database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

![Database Schema for the Project](chinook-schema.svg)

###  Overview of the Data
Let us find out the list of tables and views in our database. To this we use a special table named "SQLITE_MASTER"
Every SQLite database has an [SQLITE_MASTER](https://www.sqlite.org/faq.html) table that defines the schema for the database.  We can get access to table and index names by doing a SELECT on the "SQLITE_MASTER"

In [2]:
%%sql
SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ('table', 'view')

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We have 11 tables and no views

In [3]:
%%sql
SELECT *
    FROM album 
    LIMIT 3


 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2


## Getting an overview of employee customer relationship

In [4]:
%%sql
SELECT 
        e.first_name || " " || e.last_name employee_name,
        c.first_name || " " || c.last_name customer_name,
        SUM(i.total) total_purchased
        FROM customer c
    INNER JOIN employee e ON e.employee_id = c.support_rep_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY c.customer_id
    LIMIT 3


 * sqlite:///chinook.db
Done.


employee_name,customer_name,total_purchased
Jane Peacock,Luís Gonçalves,108.89999999999998
Steve Johnson,Leonie Köhler,82.17
Jane Peacock,François Tremblay,99.99


## Selecting Albums to Purchase
<p>Considering our target market as USA, we are interested in finding out which genres are in very high demand. To do this, we will need to extract the genres that sells the most tracks in the USA</p>
<p>For this we will need to join the following tables that can needed information for proper recommendation</p>
<ol>
    <li>Invoice Line</li>
    <li>Track</li>
    <li>Genre</li>
    <li>Invoice</li>
</ol>

In [5]:
%%sql
WITH 
    usa_sales AS
    (
        SELECT 
            invoice_id,
            billing_country country_sales
        FROM invoice
        WHERE country_sales = 'USA'
    ),
    usa_track_sales AS
    (
        SELECT
            ivl.invoice_id,
            track_id,
            SUM(quantity) track_sold
        FROM invoice_line ivl
        INNER JOIN usa_sales usl ON usl.invoice_id = ivl.invoice_id
        GROUP BY track_id        
    ),
    track_genre AS
    (
        SELECT
            t.track_id,
            t.name track_name,
            t.genre_id,
            g.name genre_name
        FROM track t
        INNER JOIN genre g ON g.genre_id = t.genre_id
        GROUP BY t.genre_id
    )
SELECT 
        tg.genre_name,
        usts.track_sold track_sold
    FROM track_genre as tg
    INNER JOIN usa_track_sales usts ON usts.track_id = tg.track_id

 * sqlite:///chinook.db
Done.


genre_name,track_sold
Blues,1
Latin,1
Pop,1
Electronica/Dance,1
Hip Hop/Rap,1
Alternative,1


In [6]:
%%sql
WITH 
    usa_sales AS
    (
        SELECT 
            invoice_id,
            billing_country country_sales
        FROM invoice
        WHERE country_sales = 'USA'
    ),
    usa_track_sales AS
    (
        SELECT
            ivl.invoice_id,
            track_id,
            ivl.quantity
        FROM invoice_line ivl
        INNER JOIN usa_sales usl ON usl.invoice_id = ivl.invoice_id        
    ),
    track_genre AS
    (
        SELECT
            t.track_id,
            t.name track_name,
            t.genre_id,
            g.name genre_name
        FROM track t
        INNER JOIN genre g ON g.genre_id = t.genre_id
    )
SELECT 
        tg.genre_name,
        SUM(usts.quantity) quantity,
        CAST(SUM(usts.quantity) AS FLOAT) / (SELECT SUM(quantity) FROM usa_track_sales)*100 AS Percentage
    FROM track_genre AS tg
INNER JOIN usa_track_sales usts ON usts.track_id = tg.track_id
GROUP BY tg.genre_id
ORDER BY quantity DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre_name,quantity,Percentage
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


In USA, the three most popular genre with large volume of sales are the Rock (53.4%), Alternative (12.4%) & Punk and Metal (11.8%).

But, selecting first three albums from the list of four according to the most popular genre:
* Red Tone - Punk
* Slim Jim Bites - Blues
* Meteor and the Girls - Pop

## Analyzing Employees Sales Performance

In [7]:
%%sql
WITH 
    customer_total_purchase AS 
    (
        SELECT 
            c.*,
            i.total
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = i.customer_id
    )
SELECT 
    e.first_name || " " || e.last_name employee_name,
    e.title,
    SUM(ctp.total) customer_purchases
FROM employee e
LEFT JOIN customer_total_purchase ctp ON e.employee_id = ctp.support_rep_id
GROUP BY e.employee_id
ORDER BY customer_purchases DESC

 * sqlite:///chinook.db
Done.


employee_name,title,customer_purchases
Jane Peacock,Sales Support Agent,98898.03000000729
Margaret Park,Sales Support Agent,94188.6000000066
Steve Johnson,Sales Support Agent,84769.74000000287
Andrew Adams,General Manager,None
Nancy Edwards,Sales Manager,None
Michael Mitchell,IT Manager,None
Robert King,IT Staff,None
Laura Callahan,IT Staff,None


As expected, sales were made only by Sales Support Agents. The remaining employees with no sales are the General manager, Sales manager, IT manager and IT staff whose job functions ordinarily does not directly related with interaction with customers.  Jane Peaccok is the Sales Support Agent with highest sales of almost $99,000

## Analyzing Sales by Country

#### Preparing the Initial code
Identifying all the countries with just one customer

In [8]:
%%sql
select CASE
            WHEN COUNT(customer_id) > 1 THEN country
            ELSE 'others'
            end as country_new,
            count(customer_id) customer
            from customer
    group by country

 * sqlite:///chinook.db
Done.


country_new,customer
others,1
others,1
others,1
others,1
Brazil,5
Canada,8
others,1
Czech Republic,2
others,1
others,1


#### Combining the countries with just one customer together into 'others'

In [9]:
%%sql
WITH country_customer AS
    (select CASE
            WHEN COUNT(customer_id) > 1 THEN country
            ELSE 'others'
            end as country_new,
            count(customer_id) customer
            from customer
    group by country)
    
SELECT 
    country_new, 
    SUM(customer) no_of_customer 
FROM country_customer 
GROUP BY country_new

 * sqlite:///chinook.db
Done.


country_new,no_of_customer
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Portugal,2
USA,13
United Kingdom,3
others,15


#### Combining the countries with just one customer together in 'others' (2)

In [10]:
%%sql
SELECT 
        country_new, 
        SUM(customer) no_of_customer 
    FROM (
            SELECT CASE
                WHEN COUNT(customer_id) > 1 THEN country
                ELSE 'others'
                END AS country_new,
                count(customer_id) customer
            FROM customer
            GROUP BY country
         ) 
GROUP BY country_new

 * sqlite:///chinook.db
Done.


country_new,no_of_customer
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Portugal,2
USA,13
United Kingdom,3
others,15


#### The Solution

In [11]:
%%sql
WITH summary AS
(SELECT 
    CASE
        WHEN COUNT(DISTINCT c.customer_id) > 1 THEN c.country
        ELSE 'others'
        END AS Country,
        COUNT(DISTINCT c.customer_id) total_customer,
        SUM(il.unit_price) total_sales,
        COUNT( DISTINCT i.invoice_id) total_order
    FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
GROUP BY Country)

SELECT 
    Country,
    SUM(total_customer) customer_total,
    SUM(total_sales) total_Sales,
    SUM(total_sales)/SUM(total_order) average_order,
    SUM(total_sales)/SUM(total_customer) average_sale_customer
FROM summary
GROUP BY Country 
ORDER BY total_customer DESC

 * sqlite:///chinook.db
Done.


Country,customer_total,total_Sales,average_order,average_sale_customer
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
others,15,1094.9399999999994,7.448571428571424,72.99599999999995


## Albums Vs Individual Tracks

Exploring the relevant tables to this analysis

In [12]:
%%sql
select * from invoice_line where invoice_id = 1

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


In [13]:
%%sql
select * from invoice_line where track_id = 1158

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
3742,482,1158,0.99,1


In [14]:
%%sql
SELECT
    il.invoice_id,
    t.album_id,
    t.track_id
FROM invoice_line il
LEFT JOIN track t ON il.track_id = t.track_id
order by t.album_id
limit 5


 * sqlite:///chinook.db
Done.


invoice_id,album_id,track_id
21,1,13
23,1,1
23,1,6
23,1,7
23,1,8


In [15]:
%%sql
SELECT
    album_id,
    track_id
FROM track
order by album_id
limit 5

 * sqlite:///chinook.db
Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9


In [16]:
%%sql
SELECT
    track_id
FROM track t

EXCEPT

SELECT
    track_id
FROM invoice_line

 * sqlite:///chinook.db
Done.


track_id
99
101
104
106
107
111
112
113
114
115


#### The Solution

In [17]:
%%sql 
WITH 
    track_not_bought AS
        (
            SELECT
                track_id
            FROM track t

        EXCEPT

            SELECT
                track_id
            FROM invoice_line
        ),
        
    album_purchase AS
        (
            SELECT
                il.invoice_id,
                t.album_id,
                t.track_id,
                CASE
                        WHEN t.album_id IN (
                                            SELECT  DISTINCT album_id
                                                FROM track t2
                                            INNER JOIN track_not_bought tnb ON  tnb.track_id = t2.track_id
                                            )
                    THEN 'album_not_Purchased'
                    ELSE 'album_purchased'
                    END AS album_cat
                FROM invoice_line il
                INNER JOIN track t ON il.track_id = t.track_id
                GROUP BY il.invoice_id, t.album_id
        ),
        
  
    invoice_album_cat AS
    (
        SELECT 
            album_cat, 
            count(invoice_id)
        FROM album_purchase
        GROUP BY invoice_id
    )

    
SELECT 
    album_cat, 
    COUNT(album_cat) invoice_count,
    CAST(COUNT(album_cat) AS FLOAT) / (SELECT COUNT(*) FROM invoice)*100 Perc
from invoice_album_cat
GROUP by album_cat

 * sqlite:///chinook.db
Done.


album_cat,invoice_count,Perc
album_not_Purchased,112,18.241042345276874
album_purchased,502,81.75895765472313


In [18]:
%%sql
SELECT
    il.invoice_id,
    t.album_id,
    count(distinct il.track_id) track_count,
    CASE
         WHEN t.album_id || ", " || count(distinct il.track_id) IN (
                                            SELECT 
                                                album_id || ", " || COUNT(track_id) track_count
                                            FROM track
                                            GROUP BY album_id
                                            )
        THEN "album_purchased"
    
    ELSE "album_not_purchased"
    END AS album_cat
FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
group by il.invoice_id, t.album_id
having il.invoice_id = 4

 * sqlite:///chinook.db
Done.


invoice_id,album_id,track_count,album_cat
4,58,1,album_not_purchased
4,67,1,album_not_purchased
4,146,1,album_not_purchased
4,153,1,album_not_purchased
4,203,1,album_not_purchased
4,207,1,album_not_purchased
4,260,1,album_purchased
4,314,1,album_not_purchased


In [19]:
%%sql
WITH 
    album_purchase AS
    (
        SELECT
        il.invoice_id,
        t.album_id,
        count(distinct il.track_id) track_count,
        CASE
            WHEN t.album_id || ", " || count(distinct il.track_id) IN (
                                                SELECT 
                                                    album_id || ", " || COUNT(track_id) track_count
                                                FROM track
                                                GROUP BY album_id
                                                ) AND COUNT(t.album_id) > 1
            THEN "album_purchased"
            ELSE "album_not_purchased"
            END AS album_cat
    FROM track t
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    group by il.invoice_id, t.album_id
    ORDER BY album_id
    ),

    invoice_album_cat AS
    (
        SELECT 
            album_cat, 
            count(invoice_id)
        FROM album_purchase
        GROUP BY invoice_id
    )

SELECT 
    album_cat, 
    COUNT(album_cat) invoice_count,
    CAST(COUNT(album_cat) AS FLOAT) / (SELECT COUNT(*) FROM invoice)*100 Perc
from invoice_album_cat
GROUP by album_cat

 * sqlite:///chinook.db
Done.


album_cat,invoice_count,Perc
album_not_purchased,502,81.75895765472313
album_purchased,112,18.241042345276874
